## HealthBite - Recomendador de recetas - Proceso Completo

### **Objetivo**

Este notebook describe, paso a paso, cómo funciona la aplicación demo de HealthBite. El objetivo es explicar las funciones de la aplicación y el orden del proceso para entender de forma clara el flujo completo. HealthBite es una aplicación capaz de  detectar ingredientes disponibles a partir de una foto y, a la vez, sugerir recetas que ayuden a cubrir posibles deficiencias nutricionales inferidas de descripciones anímicas o físicas del usuario teniendo en cuenta el contexto de los ingredientes de la nevera.

### **Flujo del proceso**

Resumidamente, los pasos principales del recomendador de recetas son:

`Entrada de inputs necesarios`:
- Foto de la nevera
- Descripción anímica o del estado físico

`Flujo del proceso`
- 1) La aplicación procesará con un **modelo YOLO entrenado y finetuneado con imágenes de nevera** la foto de la nevera e identificará la lista de ingredientes disponibles que tiene el usuario
- 2) Por otro lado, la aplicación también procesará con un **modelo NLP entrenado con un dataset sintético de detección de síntomas en base a una descripción(basado en páginas y referencias médicas)** para detectar el síntoma en base a la descripción proporcionado por el usuario. Después de identificar el síntoma, también asociará el síntoma con posibles deficiencias nutriocionales e identificará en qué ingredientes se puede suplir la deficiencia nutricional. 
- 3) Después de obtener las predicciones de los pasos 1 y 2, la aplicación usará tanto el síntoma como la lista de ingredientes disponibles y construirá un sistema con métricas de puntuación para obtener las top 10 recetas con mayor puntuación de un dataset de 2+ millones de recetas (previamente traducidas al español con un **modelo NLP arquitectura MarianMT especializado en traducción del inglés al español**). Algunos de los aspectos priorizados por el sistema de puntuación pueden ser como si la receta cubre la deficiencia nutriocional, si la receta se puede hacer con los ingredientes disponibles en la nevera, etc.
- 4) Por último, habrá un modelo LLM (Large Language Model) que servirá como juez y se aplicará RAG (Retrieval-Augmented Generation) para decidir 3 recetas finales de las top 10 obtenidas en el paso 3. El modelo LLM seguirá una serie de instrucciones para tomar la decisión final. Una vez el LLM haya decidido las tres recetas finales, las devolverá como output y las comunicará con el usuario con el título, los ingredientes necesarios, y una breve explicación de por qué se han escogido esas recetas.

`Nota adicional`: Las funciones finales para el prototipo aplicación pueden cambiar ligeramente para que los inputs sean interactivos con la aplicación en sí. Es decir, que el input pueda ser dinámico en base a lo que el usuario teclee en la aplicación.

## Imports

In [1]:
# ---- imports ----
import os, json, uuid
from datetime import datetime
import numpy as np
import pandas as pd
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from ultralytics import YOLO
import unicodedata
import re
import requests
import ast
from sklearn.preprocessing import MultiLabelBinarizer
from scipy import sparse

c:\Users\oscar.xu\AppData\Local\anaconda3\envs\tfm_env_final\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1) Modelo YOLO detector de ingredientes + modelo NLP detector de síntomas

En primer lugar, prepararemos funciones normalizadoras que ayudarán a procesar el texto y mantener un estándar consistente.

In [2]:
# ------------------ Sección de funciones normalizadoras de texto -----------------
def _strip_accents(s: str) -> str:
    """
    Función normalizadora que elimina acentos y carácteres no estándares
    Ejemplos:
    "Niño"    → "Nino"
    "canción"   → "cancion

    Parámetros
    ----------
    s : str
        Texto de entrada

    Devuelve
    -------
    str
        Cadena sin acentos/diacríticos y normalizada (NFKC).
    """
    s = unicodedata.normalize("NFD", s)
    s = "".join(ch for ch in s if unicodedata.category(ch) != "Mn")
    return unicodedata.normalize("NFKC", s)

def _norm(s: str) -> str:
    """
    Normaliza texto para comparaciones quitando espacios y pasando a minúsculas.
    En esta función se reutiliza la función strip_accents definido previamente

    Pasos:
      - strip(): recorta espacios iniciales/finales.
      - lower(): minúsculas.
      - _strip_accents(): función anterior definida para quitar acentos y carácteres no estándares
      - re.sub(r"\\s+", " ", ...): reemplaza cualquier secuencia de espacios en blanco
        (incluye tabs/saltos de línea) por un único espacio.

    Ejemplos
    --------
    _norm("  Niño   Feliz\\n")
    'nino feliz'
    _norm("  CANCIÓN  ")
    'cancion'

    Parámetros
    ----------
    s : str
        Texto de entrada.

    Devuelve
    -------
    str
        Texto normalizado para comparaciones futuras.
    """
    s = _strip_accents((s or "").strip().lower())
    s = re.sub(r"\s+", " ", s)
    return s

### 1.1) Modelo NLP

In [3]:
# ---- NLP wrapper ----
class SymptomClassifier:
    """
    Una clase llamada "SymptomClassifier" que contiene el modelo de clasificación de texto NLP transformers.
    El modelo predice un síntoma a partir de la descripción del usuario 

    SymptomClassifier:
      - Carga `AutoTokenizer` y `AutoModelForSequenceClassification` desde `model_dir`.
      - Selecciona automáticamente el dispositivo, utiliza GPU si está disponible (para que sea más rápido)
      - Tiene un mapeo para traducir ids ↔ etiquetas (es decir, la etiqueta está en número pero hay un mapeo que lo traduce a algo "legible")

    Parámetros
    ----------
    model_dir : str
        Ruta a la carpeta del modelo entrenado
    labels_path : str
        Ruta a `symptom2id.json`, el documento de mapeo
    device : str o None, opcional
        Dispositivo sobre el que ejecutar el modelo (p. ej., "cuda" / "cpu").
        Si es None, se detecta automáticamente.
    max_length : int, opcional
        Longitud máxima de secuencia para el tokenizador (truncación si se excede).

    Ejemplo
    -------
    El modelo devuelve el síntoma predecido y la confianza de predicción.
    >>> clf = SymptomClassifier("../mi_modelo")
    >>> label, conf = clf.predict_one("Me siento muy cansado últimamente")
    ('fatiga', 0.842)
    """
    
    def __init__(self, model_dir, labels_path, device=None, max_length=256):
        self.tokenizer = AutoTokenizer.from_pretrained(model_dir)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_dir)
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)
        self.max_length = max_length

        if os.path.exists(labels_path):
            with open(labels_path, "r", encoding="utf-8") as f:
                mapping = json.load(f)
            if all(not str(k).isdigit() for k in mapping.keys()):   
                self.label2id = {str(k): int(v) for k, v in mapping.items()}
                self.id2label = {v: k for k, v in self.label2id.items()}
            else:                                                    
                self.id2label = {int(k): str(v) for k, v in mapping.items()}
                self.label2id = {v: k for k, v in self.id2label.items()}
        else:
            id2label = getattr(self.model.config, "id2label", None)
            if id2label:
                self.id2label = {int(k): v for k, v in id2label.items()}
            else:
                self.id2label = {i: f"LABEL_{i}" for i in range(self.model.config.num_labels)}

# --- Parte de predicción -----
    @torch.inference_mode()
    def predict_one(self, text: str):
        enc = self.tokenizer([text], padding=True, truncation=True,
                             max_length=self.max_length, return_tensors="pt").to(self.device)
        self.model.eval()
        out = self.model(**enc)
        probs = torch.softmax(out.logits, dim=1).detach().cpu().numpy()[0]
        pred_id = int(np.argmax(probs))
        label = self.id2label.get(pred_id, f"LABEL_{pred_id}")
        conf = float(probs[pred_id])
        return label, conf

Instanciamos el modelo NLP a continuación:


In [4]:
# ---- instantiate models (adjust paths) ----
model_dir = r"..\models\NLP\NLP best model" 
labels_path = r"..\models\NLP\NLP best model\symptom2id.json" 

sym_clf = SymptomClassifier(model_dir, labels_path=labels_path)

A continuación realizamos un ejemplo de como funcionaría, probamos con cualquier oración describiendo nuestro estado físico o como nos sentimos

In [5]:
sym_clf.predict_one('Llevo días que me encuentro cansado')

('fatiga', 0.4192424416542053)

### 1.2) Modelo YOLO

En primer lugar, cargamos los weights del mejor modelo que obtuvimos entrenando con imágenes de neveras. Para mayor detalle del proceso de entrenamiento, véase el notebook "2) Entrenamiento del modelo". 

In [6]:
yolo_weights = r"..\models\YOLO\runs\YOLO best model\train 3 - 100 epochs with synthetic\weights\best.pt"
yolo = YOLO(yolo_weights)

A continuación, construimos una función para detectar los objetos disponibles en la nevera utilizando el modelo YOLO entrenado.

In [7]:
def detect_ingredients_list(image_path: str, conf_threshold: float = 0.5):
    """
    Detecta ingredientes en una imagen usando un modelo YOLO y 
    devuelve una lista de nombres de ingredientes.

    Flujo:
      1) Ejecuta el detector: `yolo(image_path)[0]` -> coge la primera imagen detectada
      2) Recorre las cajas detectadas (`res.boxes`), leyendo:
         - `cls_id`: id de clase predicha.
         - `conf`: confianza de la predicción.
      3) Filtra por `conf_threshold` (omite detecciones de baja confianza).
      4) Convierte `cls_id` a nombre humano con `res.names[cls_id]`.
      5) Normaliza el texto con `_norm` (minúsculas, sin acentos, espacios colapsados).
      6) Elimina duplicados y ordena alfabéticamente antes de devolver.

    Parámetros
    ----------
    image_path : str
        Ruta a la imagen de entrada (la foto de la nevera)
    conf_threshold : float, opcional (por defecto=0.5)
        Umbral mínimo de confianza para aceptar una detección.
        Valores más altos (p. ej., 0.25–0.5) reducen falsos positivos, si el objetivo es detectar lo máximo posible, se puede usar un umbral más bajo.

    Devuelve
    -------
    list[str]
        Lista de nombres de ingredientes

    Ejemplo
    -------
    >>> detect_ingredients_list("fridge.jpg", conf_threshold=0.3)
    ['huevo', 'leche', 'tomate']
    """
    res = yolo(image_path)[0]
    names = res.names  
    detected = []
    for i in range(len(res.boxes)):
        cls_id = int(res.boxes.cls[i].item())
        conf = float(res.boxes.conf[i].item())
        if conf < conf_threshold:
            continue
        raw = names.get(cls_id, str(cls_id))
        normed = _norm(raw)  
        detected.append(normed)
    return sorted(set(detected))


Hacemos algunos ejemplos de detección de objetos

In [8]:
#Utilizamos un threshold bajo para detectar lo máximo posible
# Probamos con imagen 1
predicted_df_1 = detect_ingredients_list(r'..\dataset\YOLO - Clean dataset\Real fridge pictures\my_fridge_1.jpg', 
                                       conf_threshold=0.2)

predicted_df_1


image 1/1 c:\Users\oscar.xu\Desktop\TFM_RecoBite\notebooks\..\dataset\YOLO - Clean dataset\Real fridge pictures\my_fridge_1.jpg: 768x576 3 Tomates, 1 Zanahoria, 1 Huevos, 2 Yogurs, 2 Aguacates, 1 Limn, 39.9ms
Speed: 3.7ms preprocess, 39.9ms inference, 1.9ms postprocess per image at shape (1, 3, 768, 576)


['aguacate', 'huevos', 'limon', 'tomate', 'yogur', 'zanahoria']

In [9]:
#Utilizamos un threshold bajo para detectar lo máximo posible
# Probamos con imagen 2
predicted_df_2 = detect_ingredients_list(r'..\dataset\YOLO - Clean dataset\Real fridge pictures\my_fridge_2.jpg', 
                                       conf_threshold=0.2)

predicted_df_2


image 1/1 c:\Users\oscar.xu\Desktop\TFM_RecoBite\notebooks\..\dataset\YOLO - Clean dataset\Real fridge pictures\my_fridge_2.jpg: 768x576 2 Tomates, 1 Zanahoria, 1 Pepino, 1 Huevos, 3 Yogurs, 1 Queso, 2 Pltanos, 2 Aguacates, 1 Limn, 1 Manzana, 5.5ms
Speed: 4.0ms preprocess, 5.5ms inference, 3.2ms postprocess per image at shape (1, 3, 768, 576)


['aguacate',
 'huevos',
 'limon',
 'manzana',
 'pepino',
 'platano',
 'queso',
 'tomate',
 'yogur',
 'zanahoria']

### 1.3) Combinación de modelos YOLO + NLP

En esta sección combinaremos las secciones anteriores y construiremos una función que combine tanto el predictor NLP como el modelo YOLO. 
El modelo NLP se ha entrenado con síntomas guardados en un dataset llamado `deficiencies_dataset`. Este dataset ha sido generado manualmente y sintéticamente consultando fuentes médicas para asegurar la certeza de la información. Dentro del dataset, tendremos información sobre posibles alimentos/ingredientes que pueden ayudar con las deficiencias nutricionales. De momento, el modelo también solo contempla las 30 clases de ingredientes que se han establecido previamente como resultado de un EDA en el notebook '0)EDA'. Adicionalmente, también son las clases con las que se ha entrenado el modelo YOLO.

In [10]:
def NLP_YOLO_predictor_function(text: str, image_path: str) -> pd.DataFrame:

    """
    Ejecuta el pipeline combinado de NLP + YOLO y devuelve un DataFrame de 1 fila 
    con los resultados de ambas predicciones.

    Flujo
    -----
    1) NLP (sym_clf.predict_one): a partir del texto del usuario, predice el
       síntoma (`predicted_symptom`) y su confianza (`confidence`).
    2) YOLO (detect_ingredients_list): a partir de la imagen, detecta los
       ingredientes disponibles en la nevera y los devuelve como lista en
       `fridge_ingredients_available`
    3) Construye un DataFrame con metadatos del run (`run_id`,
       `timestamp`, entradas usadas) y las predicciones del paso 1–2.
    4) Enriquecimiento: carga el CSV de deficiencias nutricionales yhace un merge por `predicted_symptom` 
    para obtener información de las posibles deficiencias nutricionales asociadas con el síntoma y los posibles ingredientes que ayudarían con las deficiencias.
    5) Convierte la columna `'ingredients supplying deficiency'` de string con
       separador `;` a lista limpia.

    Parámetros
    ----------
    text : str
        Descripción del usuario (síntomas/estado) para el clasificador NLP.
    image_path : str
        Ruta de la imagen (foto de la nevera) para el detector YOLO.

    Devuelve
    --------
    pd.DataFrame
        DataFrame de una fila con metadatos y los resultados de las predicciones.
    """
    
    run_id = str(uuid.uuid4())[:8]
    ts = datetime.now().isoformat(timespec="seconds")

    # Predictions
    symptom_label, conf = sym_clf.predict_one(text)
    ingredients = detect_ingredients_list(image_path, conf_threshold=0.05)

    # build one-row DF (wrap dict in a list!)
    df_predicted = pd.DataFrame([{
        "run_id": run_id,
        "timestamp": ts,
        "input_text": text,
        "image_path_used": image_path,
        "predicted_symptom": symptom_label,  # human-readable label
        "confidence": conf,
        "fridge_ingredients_available": ingredients      # stays as Python list in a single cell
    }])

    deficiency_df = pd.read_csv(r'..\dataset\Nutritional deficiency dataset\deficiencies_dataset.csv')
    deficiency_df = deficiency_df[['sintoma', 'deficiencia de nutrientes', 'disponible en ingredientes']].drop_duplicates().reset_index(drop=True)
    deficiency_df.rename(columns={'sintoma':'predicted_symptom', 'deficiencia de nutrientes': 'nutritional deficiency', 'disponible en ingredientes' : 'ingredients supplying deficiency'}, inplace=True)
    df_predicted = pd.merge(df_predicted, deficiency_df, on='predicted_symptom', how='left')  
    df_predicted['ingredients supplying deficiency'] = (df_predicted['ingredients supplying deficiency'].str.split(';').apply(lambda lst: [x.strip() for x in lst if x.strip()])  # strip spaces, drop empties
)  
    return df_predicted


In [11]:
predicted_df = NLP_YOLO_predictor_function('me duele mucho la cabeza últimamente', image_path='..\dataset\YOLO - Clean dataset\Real fridge pictures\my_fridge_2.jpg')


image 1/1 c:\Users\oscar.xu\Desktop\TFM_RecoBite\notebooks\..\dataset\YOLO - Clean dataset\Real fridge pictures\my_fridge_2.jpg: 768x576 2 Tomates, 1 Zanahoria, 1 Pepino, 1 Huevos, 3 Yogurs, 1 Queso, 2 Pltanos, 2 Aguacates, 1 Limn, 1 Manzana, 6.5ms
Speed: 3.8ms preprocess, 6.5ms inference, 3.4ms postprocess per image at shape (1, 3, 768, 576)


In [12]:
predicted_df

,run_id,timestamp,input_text,image_path_used,predicted_symptom,confidence,fridge_ingredients_available,nutritional deficiency,ingredients supplying deficiency
0,bcb4ff50,2025-08-30T19:37:59,me duele mucho la cabeza últimamente,..\dataset\YOLO - Clean dataset\Real fridge pi...,dolor de cabeza,0.5049,"[aguacate, huevos, limon, manzana, pepino, pla...",magnesio; riboflavina (b2); potasio,"[aguacate, platano, patata, tomate, calabacine..."


## 2) Algoritmo de recomendación

Una vez hemos construido la función que combina tanto el modelo NLP con el modelo YOLO, se procede a construir el algoritmo de recomendación que funciona de la siguiente manera:
- Se carga el set de datos de recetas traducidas del inglés al español (para más detalle, véase "4) Traducción del dataset de recetas"). El dataset tiene más de 2 millones de recetas
- Se definen una lista de métricas que evalua diferentes aspectos de cada receta en el set de datos de recetas para darle puntuación a cada receta. 
- Después, se utiliza la información obtenida tanto de la foto de nevera (ingredientes disponibles en la nevera) así como el síntoma detectado a partir de la descripción del usuario para rankear las recetas y obtener las 10 recetas con mayor puntuación.


In [13]:
# Cargamos primero el dataset
translated_recipes = pd.read_csv(r'..\dataset\Recipes dataset\recipes_dataset_translated_ingredients.csv')
translated_recipes.head()

,Unnamed: 0,title,ingredients,directions,link,source,NER,NER_terms,NER_terms_es
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"['brown sugar', 'milk', 'vanilla', 'nuts', 'bu...","['brown sugar', 'milk', 'vanilla', 'nuts', 'bu...","['azúcar moreno', 'Leche', 'vainilla', 'frutos..."
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"['beef', 'chicken breasts', 'cream of mushroom...","['beef', 'chicken breasts', 'cream of mushroom...","['Carne vacuno', 'Carne pollo', 'Champiñones',..."
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"['frozen corn', 'cream cheese', 'butter', 'gar...","['frozen corn', 'cream cheese', 'butter', 'gar...","['Maíz congelado', 'Queso', 'Mantequilla', 'aj..."
3,3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"['chicken', 'chicken gravy', 'cream of mushroo...","['chicken', 'chicken gravy', 'cream of mushroo...","['Carne pollo', 'Carne pollo', 'Champiñones', ..."
4,4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"['peanut butter', 'graham cracker crumbs', 'bu...","['peanut butter', 'graham cracker crumbs', 'bu...","['Mantequilla', 'migajas de galleta graham', '..."


### 2.1) Métricas para la recomendación de recetas

Las siguientes métricas son las utilizadas para evaluar las recetas y obtener una puntuación.

**Notación:**
- (R): conjunto de ingredientes de la **receta**
- (F): conjunto de ingredientes de la **nevera** (detectados por YOLO).
- (H): conjunto de **ingredientes que ayudan** a cubrir las deficiencias relacionadas con el síntoma predecido (NLP)
- (T): número de tokens/palabras del texto de instrucciones \(s\).

---
`1) Similitud de Jaccard (Receta vs Nevera)`

**Fórmula:**

$$
\text{Jaccard}(R,F) \;=\; \frac{|R \cap F|}{|R \cup F|}
$$

**En palabras:** Es una métrica que mide el solapamiento de los ingredientes necesarios para la receta y los disponibles en la nevera. Se divide el solapamiento por la unión de ingredientes de ambos.

**Rango:** 0–1 (**cuanto más alto mejor**)

---

`2) Penalización por faltantes`

**Fórmula:**

$$
\text{MissingPen}(R,F) \;=\; \frac{|R \setminus F|}{|R|}
$$

**En palabras:** cuenta los ingredientes de la receta que faltan en la nevera y normaliza por el total de la receta.

**Rango:** 0–1 (**cuanto más bajo es mejor**)  

---

`3) Cobertura de deficiencias`

**Fórmula:**

$$
\text{DefCov}(R,H) \;=\;
\begin{cases}
\dfrac{|R \cap H|}{|H|}, & \text{si } |H|>0 \\[6pt]
\end{cases}
$$

**En palabras:** Calcula la cobertura de las deficiencias nutricionales. Mide de los ingredientes que ayudan a cubrir las deficiencias (H), cuántos aparecen en la receta (R). Se normaliza por (H).

**Rango:** 0–1 (**cuanto más alto mejor**)

---

`4) Esfuerzo (longitud de instrucciones)`

**Fórmula:**

$$
\text{Effort}(T) \;=\; \frac{T - T_{\min}}{\max\!\left(1,\; T_{\max} - T_{\min}\right)}
$$

**En palabras:** Se asume que las recetas con instrucciones más largas implican mayor esfuerzo. Por lo tanto, se normaliza la cantidad de tokens de texto de las recetas con min-max de forma que la receta más corta sea \(0\) y la más larga \(1\). Así podemos medir o quantificar el esfuerzo de una receta.

**Rango:** 0–1 (**cuanto más bajo mejor**)


Transformamos lo definido a código. Se calculan las métricas definidas anteriormente.

In [14]:
def _parse_terms(x):
    """
    Normaliza y convierte distintos posibles formatos de términos de ingredientes a una lista de strings
    en minúsculas y sin espacios extra.

    Inputs aceptados:
      - list: se normaliza cada elemento con str().strip().lower().
      - str que representa una lista: intenta parsear con ast.literal_eval (p. ej. "['tomate','leche']").
      - str con términos separados por comas: "tomate, leche, huevo".


    Ejemplos
    --------
    >>> _parse_terms(["Tomate", "  Leche  ", ""])
    ['tomate', 'leche']
    >>> _parse_terms("['Tomate','LECHE']")
    ['tomate', 'leche']
    >>> _parse_terms("tomate, leche ,  huevo")
    ['tomate', 'leche', 'huevo']

    Devuelve
    -------
    list[str]
        Lista normalizada de términos.
    """

    if isinstance(x, list):
        return [str(t).strip().lower() for t in x if str(t).strip()]

    if isinstance(x, str):
        try:
            v = ast.literal_eval(x)
            if isinstance(v, list):
                return [str(t).strip().lower() for t in v if str(t).strip()]
        except Exception:
            pass
        return [t.strip().lower() for t in x.split(",") if t.strip()]
    return []

def compute_recipe_scores(recipes_df, fridge_list, deficiency_ingredients_list=None):
    """
    Esta función calcula las siguientes métricas:
    1) Jaccard score
    2) Penalización por faltantes (missing_penalization)
    3) Cobertura de deficiencias (DRC_coverage)
    4) Esfuerzo (effort)


    Parámetros
    --------
    recipes_df : pd.DataFrame que contendrá los ingredientes de la receta y también las instrucciones
    fridge_list : list[str]
        Ingredientes detectados en la nevera
    deficiency_ingredients_list : list[str]
        Ingredientes que ayudan a cubrir deficiencias

    Comentarios adicionales
    -----------------------
    - Convierte 'NER_terms_es' (la columna del df que contiene los ingredientes necesarios de la receta) a listas limpias con `_parse_terms`.
    - Usa MultiLabelBinarizer para crear una **matriz binaria dispersa** X (recetas × vocabulario),
      lo que permite calcular intersecciones y tamaños de conjuntos sin bucles Python (es más eficiente y más rápido que un bucle Python en 2 millones de filas).
    - El tamaño de la receta |R| se obtiene con `np.diff(X.indptr)` (traversal O(1) por fila en CSR).
    - La normalización de esfuerzo usa min–max global

    Devuelve
    -------
    pd.DataFrame
        Copia de `recipes_df` con columnas añadidas:
        ['recipe_ingredients','jaccard_score','missing_penalization',
        'DRC_coverage','Effort']
    """

     
    df = recipes_df.copy()

    # 1) Normalizamos los inputs
    fridge = set(_parse_terms(fridge_list)) if isinstance(fridge_list, (list, str)) else set()
    deficiency_ingredients_list = [s.strip().lower() 
                               for s in deficiency_ingredients_list[0].split(",")]
    deficiencies = set(_parse_terms(deficiency_ingredients_list)) if isinstance(deficiency_ingredients_list, (list, str)) else set()

    terms = df["NER_terms_es"].map(_parse_terms)

    # 2) Matriz binaria recetas×vocabulario
    mlb = MultiLabelBinarizer(sparse_output=True)  
    X = mlb.fit_transform(terms)                   
    vocab = np.array(mlb.classes_)                 
    fridge_mask = np.isin(vocab, list(fridge))
    def_mask = np.isin(vocab, list(deficiencies)) if deficiencies else np.zeros_like(vocab, dtype=bool)

    # ------------ Cálculo de métricas ----------------------
    # 1. Cálculo de jaccard score
    # inter = |R ∩ F|  → suma por fila de las columnas del vocab que están en la nevera
    inter = (X[:, fridge_mask].sum(axis=1).A1) if fridge_mask.any() else np.zeros(X.shape[0], dtype=int)

    # recipe_size = |R| -> nº de no-nulos por fila en CSR (usando indptr)
    recipe_size = np.diff(X.indptr)

    # union = |R ∪ F| = |R| + |F| - |R ∩ F|
    union = recipe_size + fridge_mask.sum() - inter
    jaccard = np.divide(inter, np.maximum(union, 1), dtype=float)

    # Cálculo de missing penalization
    num_needed = recipe_size - inter
    missing_penalization = np.divide(num_needed, np.maximum(recipe_size, 1), dtype=float)

    # Cálculo de deficiency coverage
    den = int(def_mask.sum())
    drc_abs = X[:, def_mask].sum(axis=1).A1 if den > 0 else np.zeros(X.shape[0], dtype=int)
    drc_coverage = np.divide(drc_abs, den, out=np.zeros_like(drc_abs, dtype=float), where=den > 0)
    
    # Cálculo de effort -> número effort = normalización min–max del nº de tokens en 'directions' (más largo → mayor esfuerzo)
    def _as_text(x):
        return " ".join(x) if isinstance(x, list) else str(x or "")
    tokens = df["directions"].map(_as_text).str.findall(r"\w+").str.len()
    t_min, t_max = int(tokens.min() or 0), int(tokens.max() or 0)
    effort = (tokens - t_min) / max(1, t_max - t_min)

    # Construimos el resultado df final
    df["recipe_ingredients"] = terms  
    df["inter"] = inter
    df["union"] = union
    df["jaccard_score"] = jaccard
    df["missing_penalization"] = missing_penalization
    df["DRC_abs"] = drc_abs
    df["DRC_coverage"] = drc_coverage
    df["tokens"] = tokens
    df["effort"] = effort

    return df

Tras definir la función para calcular la puntuación de cada receta, procedemos de la siguiente forma:
- Obtenemos la información necesaria de la predicción de síntoma + ingredientes disponibles en la nevera
- Se usa la función `compute_recipe_scores` para calcular la puntuación de cada receta
- Por último, se une el DataFrame de las recetas puntuadas con el DataFrame con las predicciones para un DataFrame final que es la combinación de ambos.

In [15]:
# Llamamos el df resultante de la predicción de YOLO y el modelo NLP obtenido en la sección 1 otra vez.
predicted_df.head()

,run_id,timestamp,input_text,image_path_used,predicted_symptom,confidence,fridge_ingredients_available,nutritional deficiency,ingredients supplying deficiency
0,bcb4ff50,2025-08-30T19:37:59,me duele mucho la cabeza últimamente,..\dataset\YOLO - Clean dataset\Real fridge pi...,dolor de cabeza,0.5049,"[aguacate, huevos, limon, manzana, pepino, pla...",magnesio; riboflavina (b2); potasio,"[aguacate, platano, patata, tomate, calabacine..."


In [ ]:
# Obtenemos la información necesaria para calcular las métricas
row = predicted_df.iloc[0]  
fridge_list = row['fridge_ingredients_available']
nutritional_deficiency_list = row['nutritional deficiency']
deficiency_ingredients_list = row['ingredients supplying deficiency']
symptom_list = row['predicted_symptom']

recipe_scores = compute_recipe_scores(translated_recipes,
                                  fridge_list=fridge_list,
                                  deficiency_ingredients_list=deficiency_ingredients_list)

In [17]:
final_recipe_score_df = recipe_scores.merge(
    predicted_df[['predicted_symptom', 'nutritional deficiency',
                  'ingredients supplying deficiency', 'fridge_ingredients_available']],
    how='cross'
)

In [18]:
# Miramos la puntuación de las recetas por jaccard de forma descendiente para visualizar como es el dataframe final
final_recipe_score_df.sort_values(by='jaccard_score', ascending=False).head()

,Unnamed: 0,title,ingredients,directions,link,source,NER,NER_terms,NER_terms_es,recipe_ingredients,...,jaccard_score,missing_penalization,DRC_abs,DRC_coverage,tokens,effort,predicted_symptom,nutritional deficiency,ingredients supplying deficiency,fridge_ingredients_available
1857177,1857177,Buffalo-Style Chik'n Salad,"[""2 frozen BOCA Spicy Chikn Veggie Patties"", ""...","[""Cook patties as directed on pkg."", ""; cut in...",www.kraftrecipes.com/recipes/buffalo-style-chi...,Recipes1M,"['Veggie Patties', 'celery', 'carrot', 'tomato...","['veggie patties', 'celery', 'carrot', 'tomato...","['Huevos', 'apio', 'Zanahoria', 'Tomate', 'Pep...","[huevos, apio, zanahoria, tomate, pepino, queso]",...,0.500000,0.166667,4,0.20,31,0.011494,dolor de cabeza,magnesio; riboflavina (b2); potasio,"[aguacate, platano, patata, tomate, calabacine...","[aguacate, huevos, limon, manzana, pepino, pla..."
1266234,1266234,Slimmer'S Lunch,"[""1 None hard-boiled egg, halved"", ""3.5 oz low...","[""Arrange all ingredients in an airtight lunch...",recipes-plus.com/api/v2.0/recipes/24025,Gathered,"['egg', 'cottage cheese', 'tomatoes', 'celery'...","['egg', 'cottage cheese', 'tomatoes', 'celery'...","['Huevos', 'Queso', 'Tomate', 'apio', 'Zanahor...","[huevos, queso, tomate, apio, zanahoria, remol...",...,0.454545,0.285714,4,0.20,11,0.004079,dolor de cabeza,magnesio; riboflavina (b2); potasio,"[aguacate, platano, patata, tomate, calabacine...","[aguacate, huevos, limon, manzana, pepino, pla..."
1248534,1248534,Mexican Puffs,"[""1 None avocado, diced"", ""1 None tomato, seed...","[""Preheat the oven to 425\u00b0F. Line two bak...",recipes-plus.com/api/v2.0/recipes/36281,Gathered,"['avocado', 'tomato', 'Cheddar cheese', 'pastr...","['avocado', 'tomato', 'cheddar cheese', 'pastr...","['Aguacate', 'Tomate', 'Queso', 'pastelería', ...","[aguacate, tomate, queso, pastelería, huevos, ...",...,0.454545,0.285714,5,0.25,106,0.039303,dolor de cabeza,magnesio; riboflavina (b2); potasio,"[aguacate, platano, patata, tomate, calabacine...","[aguacate, huevos, limon, manzana, pepino, pla..."
2166456,2166456,Market-Fresh Salad,"[""4 small boneless skinless chicken breasts (1...","[""Heat barbecue to medium-high heat."", ""Brush ...",www.kraftrecipes.com/recipes/market-fresh-sala...,Recipes1M,"['chicken breasts', 'Tomato', 'torn romaine le...","['chicken breasts', 'tomato', 'torn romaine le...","['Carne pollo', 'Tomate', 'Lechuga/Endivia', '...","[carne pollo, tomate, lechuga/endivia, zanahor...",...,0.454545,0.285714,5,0.25,56,0.020764,dolor de cabeza,magnesio; riboflavina (b2); potasio,"[aguacate, platano, patata, tomate, calabacine...","[aguacate, huevos, limon, manzana, pepino, pla..."
2102536,2102536,Vegie Sailboats! (--Tasty Dish--),"[""6 pickling cucumbers, cut in half lengthwise...","[""BOAT: Fill the little sailboat (ex: pickling...",www.food.com/recipe/vegie-sailboats-tasty-dish...,Recipes1M,"['pickling cucumbers', 'roma tomatoes', 'yello...","['pickling cucumbers', 'roma tomatoes', 'yello...","['Pepino', 'Tomate', 'pimientos amarillos', 'H...","[pepino, tomate, pimientos amarillos, huevos, ...",...,0.454545,0.285714,4,0.20,80,0.029663,dolor de cabeza,magnesio; riboflavina (b2); potasio,"[aguacate, platano, patata, tomate, calabacine...","[aguacate, huevos, limon, manzana, pepino, pla..."


In [19]:
# Seleccionamos solo columnas que nos interesen:
final_recipe_score_df = final_recipe_score_df[['title', 'ingredients', 'directions', 
                      'NER_terms_es', 'recipe_ingredients', 'jaccard_score', 
                      'missing_penalization', 'DRC_coverage', 'effort', 'predicted_symptom',
                      'nutritional deficiency', 'ingredients supplying deficiency', 'fridge_ingredients_available']]

final_recipe_score_df.head(1)

,title,ingredients,directions,NER_terms_es,recipe_ingredients,jaccard_score,missing_penalization,DRC_coverage,effort,predicted_symptom,nutritional deficiency,ingredients supplying deficiency,fridge_ingredients_available
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...","['azúcar moreno', 'Leche', 'vainilla', 'frutos...","[azúcar moreno, leche, vainilla, frutos secos,...",0.0,1.0,0.05,0.022618,dolor de cabeza,magnesio; riboflavina (b2); potasio,"[aguacate, platano, patata, tomate, calabacine...","[aguacate, huevos, limon, manzana, pepino, pla..."


### 2.2) Sistema de puntuación

Dado que no tenemos datos de interacción o preferencia de usuarios por las recetas, de momento para el prototipo y utilizaremos pesos arbitrarios decididos por lógica o contexto de negocio que luego se podrá tunear y mejorar utilizando modelos de Machine Learning para obtener los mejores pesos de las métricas de las recetas una vez haya datos de uso y preferencia del consumidor.

**Razón detrás de los pesos**

- **Jaccard** (0.5, relación positiva):
Se pone bastante peso en el solapamiento que hay entre los ingredientes necesarios para la receta y lo disponible en la nevera. 

- **Missing Penalization** (0.25, relación negativa):
Se penaliza cuando una receta requiere demasiados ingredientes no disponibles en la nevera. 
La diferencia con Jaccard, que pueden parecer similares es que Jaccard también penaliza de alguna forma los extras en la nevera (ya que se divide por la unión de ingredientes), en missing penalization no.

- **Deficiency Coverage** (0.15, relación positiva):
Se valora que los ingredientes de la receta cubra las deficiencias nutricionales asociadas al síntoma predicho en base al texto del usuario.

- **Effort** (0.10, relación negativa):
Penaliza levemente que la receta sea demasiado compleja. Se asume que si es muy larga, es compleja.

In [20]:
def rank_recipes(df,
                 w_jac=0.5, w_drc=0.25, w_mp=0.15, w_effort=0.10,
                 top_n=10):
    """
    Calcula un puntaje final por receta y devuelve los top n(parámetro) con mejor puntuación de forma descendiente.
    La puntuación máxima es de 1.

    Parámetros
    ----------
    df : pd.DataFrame
    w_jac, w_drc, w_mp, w_effort: los pesos para calcular la puntuación
    top_n: la cantidad de recetas a devolver

    Devuelve
    -------
    pd.DataFrame
        Copia de `df` ordenada por `final_score` (desc).
    """
    df = df.copy()
    df["final_score"] = (
        w_jac   * df["jaccard_score"] +
        w_drc   * df["DRC_coverage"] -
        w_mp    * df["missing_penalization"] -
        w_effort* df["effort"]
    )
    return df.sort_values("final_score", ascending=False).head(top_n)

In [21]:
#Se obtienen las top 10 mejores recetas en puntuación

top_10_recipes = rank_recipes(final_recipe_score_df)
top_10_recipes

,title,ingredients,directions,NER_terms_es,recipe_ingredients,jaccard_score,missing_penalization,DRC_coverage,effort,predicted_symptom,nutritional deficiency,ingredients supplying deficiency,fridge_ingredients_available,final_score
1857177,Buffalo-Style Chik'n Salad,"[""2 frozen BOCA Spicy Chikn Veggie Patties"", ""...","[""Cook patties as directed on pkg."", ""; cut in...","['Huevos', 'apio', 'Zanahoria', 'Tomate', 'Pep...","[huevos, apio, zanahoria, tomate, pepino, queso]",0.500000,0.166667,0.20,0.011494,dolor de cabeza,magnesio; riboflavina (b2); potasio,"[aguacate, platano, patata, tomate, calabacine...","[aguacate, huevos, limon, manzana, pepino, pla...",0.273851
2166456,Market-Fresh Salad,"[""4 small boneless skinless chicken breasts (1...","[""Heat barbecue to medium-high heat."", ""Brush ...","['Carne pollo', 'Tomate', 'Lechuga/Endivia', '...","[carne pollo, tomate, lechuga/endivia, zanahor...",0.454545,0.285714,0.25,0.020764,dolor de cabeza,magnesio; riboflavina (b2); potasio,"[aguacate, platano, patata, tomate, calabacine...","[aguacate, huevos, limon, manzana, pepino, pla...",0.244839
1248534,Mexican Puffs,"[""1 None avocado, diced"", ""1 None tomato, seed...","[""Preheat the oven to 425\u00b0F. Line two bak...","['Aguacate', 'Tomate', 'Queso', 'pastelería', ...","[aguacate, tomate, queso, pastelería, huevos, ...",0.454545,0.285714,0.25,0.039303,dolor de cabeza,magnesio; riboflavina (b2); potasio,"[aguacate, platano, patata, tomate, calabacine...","[aguacate, huevos, limon, manzana, pepino, pla...",0.242985
1266234,Slimmer'S Lunch,"[""1 None hard-boiled egg, halved"", ""3.5 oz low...","[""Arrange all ingredients in an airtight lunch...","['Huevos', 'Queso', 'Tomate', 'apio', 'Zanahor...","[huevos, queso, tomate, apio, zanahoria, remol...",0.454545,0.285714,0.20,0.004079,dolor de cabeza,magnesio; riboflavina (b2); potasio,"[aguacate, platano, patata, tomate, calabacine...","[aguacate, huevos, limon, manzana, pepino, pla...",0.234008
1769104,Mediterranean English Muffin Sandwich,"[""12 small carrot, grated"", ""12 small cucumber...","[""Mix the carrots, cucumbers and tomatoes toge...","['Zanahoria', 'Pepino', 'Tomate', 'hummus', 'Q...","[zanahoria, pepino, tomate, hummus, queso, agu...",0.454545,0.285714,0.20,0.019281,dolor de cabeza,magnesio; riboflavina (b2); potasio,"[aguacate, platano, patata, tomate, calabacine...","[aguacate, huevos, limon, manzana, pepino, pla...",0.232488
666717,Turkey Cobb Salad,"[""3 oz. Backyard Grill turkey breast, diced"", ...","[""Layer lettuce, tomatoes, cucumber, egg and d...","['Carne pavo', 'Tomate', 'Queso', 'Pepino', 'H...","[carne pavo, tomate, queso, pepino, huevos]",0.400000,0.200000,0.25,0.007045,dolor de cabeza,magnesio; riboflavina (b2); potasio,"[aguacate, platano, patata, tomate, calabacine...","[aguacate, huevos, limon, manzana, pepino, pla...",0.231796
2102536,Vegie Sailboats! (--Tasty Dish--),"[""6 pickling cucumbers, cut in half lengthwise...","[""BOAT: Fill the little sailboat (ex: pickling...","['Pepino', 'Tomate', 'pimientos amarillos', 'H...","[pepino, tomate, pimientos amarillos, huevos, ...",0.454545,0.285714,0.20,0.029663,dolor de cabeza,magnesio; riboflavina (b2); potasio,"[aguacate, platano, patata, tomate, calabacine...","[aguacate, huevos, limon, manzana, pepino, pla...",0.231449
1086938,Ww 5 Points - Shrimp Louis Salad,"[""1 cup cottage cheese"", ""1/4 cup tomato juice...","[""In blender combine cottage cheese, tomato ju...","['Queso', 'Tomate', 'Huevos', 'mostaza', 'Pepi...","[queso, tomate, huevos, mostaza, pepino, gamba...",0.416667,0.375000,0.30,0.025213,dolor de cabeza,magnesio; riboflavina (b2); potasio,"[aguacate, platano, patata, tomate, calabacine...","[aguacate, huevos, limon, manzana, pepino, pla...",0.224562
1684146,Katie's Favorite Salad,"[""mesclun"", ""1 medium diced tomato"", ""1 medium...","[""Toss ingredients and serve.""]","['Tomate', 'Pepino', 'Aguacate', 'Nueces', 'Qu...","[tomate, pepino, aguacate, nueces, queso]",0.400000,0.200000,0.20

## 3) LLM as a judge

En esta sección, se utilizará un modelo LLM (Large Language Model) que servirá como juez y se aplicará RAG (Retrieval-Augmented Generation) para decidir 3 recetas finales de las top 10 obtenidas en el paso 3. El modelo LLM seguirá una serie de instrucciones para tomar la decisión final. Una vez el LLM haya decidido las tres recetas finales, las devolverá como output y las comunicará con el usuario con el título, los ingredientes necesarios, y una breve explicación de por qué se han escogido esas recetas. En este caso se está utilizando el modelo `llama3.1:8b` de Ollama para LLM.

Las instrucciones que recibirá el LLM serán las siguientes:

**Como `system_prompt` recibirá:**

- Usa EXCLUSIVAMENTE la información del contexto proporcionado, no inventes nada
- Tienes que verificar si la receta de verdad es para una comida. Es decir, descarta recetas de cremas, mascarillas y otras cosas que usan ingredientes de comida pero realmente no son para la nutrición
- Tienes que verificar que la receta es realista para hacer, es decir que el usuario tiene bastantes ingredientes en la nevera de los requeridos por la receta
- Tienes que verificar si la receta de verdad cubriría con la posible deficiencia nutricional asociada al síntoma detectado"""


**Como `user_prompt` recibirá:**

Acuérdate que no puedes usar ninguna información que no existe o inventártelo.
El texto que me tienes devolver tiene que seguir el siguiente formato:
- Primero, me tienes que hablar directamente, no uses tercera persona
- Segundo, dime que síntoma me detectas en base a mi texto 
          y que deficiencias nutriocionales puede que padezca en base a {deficiency_list}. (Asegúrate de mencionar que siempre es mejor visitar el médico si es grave) 
- Tercero, dime los ingredientes detectados en mi nevera ({available_ingredients}) 
- En base a esta información, recomiendame recetas:

Para cada receta elegida, tienes que explicarme:
- El nombre de la receta {recipe_title}
- Los ingredientes necesarios de la receta {recipe_ingredients} (traducidos al español)
- explica en 1–2 frases por qué ayuda al síntoma (cita qué alimentos o ingredientes cubren las deficiencias)
- sugiere algunas sustituciones lógicas o sugiere qué ingredientes el usuario podría saltarse de la receta. Solo si el usuario no tiene el ingrediente de la nevera.
          No hace falta mencionar sustituciones de ingredientes comunes que todo el mundo tendría como agua, sal, aceite, etc.
- Describir las instrucciones y los pasos de la receta (información disponible en {recipe_instructions})

In [22]:
## Paso 1) Obtenemos las top 10 recetas según el sistema de puntuación que hemos construido
top_10_recipes

,title,ingredients,directions,NER_terms_es,recipe_ingredients,jaccard_score,missing_penalization,DRC_coverage,effort,predicted_symptom,nutritional deficiency,ingredients supplying deficiency,fridge_ingredients_available,final_score
1857177,Buffalo-Style Chik'n Salad,"[""2 frozen BOCA Spicy Chikn Veggie Patties"", ""...","[""Cook patties as directed on pkg."", ""; cut in...","['Huevos', 'apio', 'Zanahoria', 'Tomate', 'Pep...","[huevos, apio, zanahoria, tomate, pepino, queso]",0.500000,0.166667,0.20,0.011494,dolor de cabeza,magnesio; riboflavina (b2); potasio,"[aguacate, platano, patata, tomate, calabacine...","[aguacate, huevos, limon, manzana, pepino, pla...",0.273851
2166456,Market-Fresh Salad,"[""4 small boneless skinless chicken breasts (1...","[""Heat barbecue to medium-high heat."", ""Brush ...","['Carne pollo', 'Tomate', 'Lechuga/Endivia', '...","[carne pollo, tomate, lechuga/endivia, zanahor...",0.454545,0.285714,0.25,0.020764,dolor de cabeza,magnesio; riboflavina (b2); potasio,"[aguacate, platano, patata, tomate, calabacine...","[aguacate, huevos, limon, manzana, pepino, pla...",0.244839
1248534,Mexican Puffs,"[""1 None avocado, diced"", ""1 None tomato, seed...","[""Preheat the oven to 425\u00b0F. Line two bak...","['Aguacate', 'Tomate', 'Queso', 'pastelería', ...","[aguacate, tomate, queso, pastelería, huevos, ...",0.454545,0.285714,0.25,0.039303,dolor de cabeza,magnesio; riboflavina (b2); potasio,"[aguacate, platano, patata, tomate, calabacine...","[aguacate, huevos, limon, manzana, pepino, pla...",0.242985
1266234,Slimmer'S Lunch,"[""1 None hard-boiled egg, halved"", ""3.5 oz low...","[""Arrange all ingredients in an airtight lunch...","['Huevos', 'Queso', 'Tomate', 'apio', 'Zanahor...","[huevos, queso, tomate, apio, zanahoria, remol...",0.454545,0.285714,0.20,0.004079,dolor de cabeza,magnesio; riboflavina (b2); potasio,"[aguacate, platano, patata, tomate, calabacine...","[aguacate, huevos, limon, manzana, pepino, pla...",0.234008
1769104,Mediterranean English Muffin Sandwich,"[""12 small carrot, grated"", ""12 small cucumber...","[""Mix the carrots, cucumbers and tomatoes toge...","['Zanahoria', 'Pepino', 'Tomate', 'hummus', 'Q...","[zanahoria, pepino, tomate, hummus, queso, agu...",0.454545,0.285714,0.20,0.019281,dolor de cabeza,magnesio; riboflavina (b2); potasio,"[aguacate, platano, patata, tomate, calabacine...","[aguacate, huevos, limon, manzana, pepino, pla...",0.232488
666717,Turkey Cobb Salad,"[""3 oz. Backyard Grill turkey breast, diced"", ...","[""Layer lettuce, tomatoes, cucumber, egg and d...","['Carne pavo', 'Tomate', 'Queso', 'Pepino', 'H...","[carne pavo, tomate, queso, pepino, huevos]",0.400000,0.200000,0.25,0.007045,dolor de cabeza,magnesio; riboflavina (b2); potasio,"[aguacate, platano, patata, tomate, calabacine...","[aguacate, huevos, limon, manzana, pepino, pla...",0.231796
2102536,Vegie Sailboats! (--Tasty Dish--),"[""6 pickling cucumbers, cut in half lengthwise...","[""BOAT: Fill the little sailboat (ex: pickling...","['Pepino', 'Tomate', 'pimientos amarillos', 'H...","[pepino, tomate, pimientos amarillos, huevos, ...",0.454545,0.285714,0.20,0.029663,dolor de cabeza,magnesio; riboflavina (b2); potasio,"[aguacate, platano, patata, tomate, calabacine...","[aguacate, huevos, limon, manzana, pepino, pla...",0.231449
1086938,Ww 5 Points - Shrimp Louis Salad,"[""1 cup cottage cheese"", ""1/4 cup tomato juice...","[""In blender combine cottage cheese, tomato ju...","['Queso', 'Tomate', 'Huevos', 'mostaza', 'Pepi...","[queso, tomate, huevos, mostaza, pepino, gamba...",0.416667,0.375000,0.30,0.025213,dolor de cabeza,magnesio; riboflavina (b2); potasio,"[aguacate, platano, patata, tomate, calabacine...","[aguacate, huevos, limon, manzana, pepino, pla...",0.224562
1684146,Katie's Favorite Salad,"[""mesclun"", ""1 medium diced tomato"", ""1 medium...","[""Toss ingredients and serve.""]","['Tomate', 'Pepino', 'Aguacate', 'Nueces', 'Qu...","[tomate, pepino, aguacate, nueces, queso]",0.400000,0.200000,0.20

In [23]:
# Paso 2) Funciones de preprocesamiento para procesar los texto de los top 10 filas que hemos obtenido con rank_recipes

def safe_to_list(value):

    """
    Convierte entradas heterogéneas en una lista de strings consistente.

    Parámetros (acepta):
    -------------------
      - list o tuple: devuelve list(...) tal cual.
      - str que "parece" lista/tupla (p. ej., "['a','b']" o "('a','b')"):
          en estos casos se intenta parsear con ast.literal_eval de forma segura.
      - str simple con separadores por coma: "a, b, c" -> ["a", "b", "c"]
      - etc

    Ejemplos
    --------
    >>> safe_to_list(["Tomate", "  Leche  "])
    ['Tomate', 'Leche']
    >>> safe_to_list("['Tomate','Leche']")
    ['Tomate', 'Leche']
    """

    if isinstance(value, list):
        return value
    if isinstance(value, str):
        stripped = value.strip()
        if (stripped.startswith("[") and stripped.endswith("]")) or (stripped.startswith("(") and stripped.endswith(")")):
            try:
                parsed = ast.literal_eval(stripped)
                if isinstance(parsed, (list, tuple)):
                    return list(parsed)
            except Exception:
                pass
        return [item.strip() for item in value.split(",") if item.strip()]
    return []

def format_ingredients(value):
    """
    Normaliza y formatea ingredientes a una cadena legible

    - Internamente usa `safe_to_list` para obtener la lista homogénea.
    - Pensado para imprimir en prompts al LLM u outputs de usuario.

    Ejemplos
    --------
    >>> format_ingredients("['tomate','leche']")
    'tomate, leche'
    >>> format_ingredients(["tomate", "leche", "huevo"])
    'tomate, leche, huevo'
    """

    ingredients_list = safe_to_list(value) if value is not None else []
    return ", ".join(map(str, ingredients_list))

def format_directions(value, max_chars=900):
    """
    Unifica el formato de instrucciones de las receta en un único string legible
    y se pone un máximo de caracteres para controlar el tamaño en prompts/LLMs.

    Parámetros (acepta):
    -------------------
      - list de pasos -> se unen con espacios.
      - str que "parece" lista (p. ej. "['Paso 1','Paso 2']") -> se evalúa con
        `ast.literal_eval` y luego se unen los elementos si realmente es una lista.
      - str normal -> se devuelve tal cual (recortado si excede max_chars).

    Ejemplos
    --------
    >>> format_directions(["Corta el tomate.", "Añade la sal."])
    'Corta el tomate. Añade la sal.'
    >>> format_directions("['Corta','Mezcla','Sirve']")
    'Corta Mezcla Sirve'
    >>> format_directions("Texto largo...", max_chars=5)
    'Texto'
    """
    
    if isinstance(value, list):
        text = " ".join(map(str, value))
    elif isinstance(value, str):
        possible_list = None
        stripped = value.strip()
        if stripped.startswith("[") and stripped.endswith("]"):
            try:
                possible_list = ast.literal_eval(stripped)
            except Exception:
                possible_list = None
        text = " ".join(possible_list) if isinstance(possible_list, list) else value
    else:
        text = ""
    return text[:max_chars]


In [24]:
# Paso 3) Construir el contexto para el LLM

def make_context_from_top10(top10_df):

    """
    Construye un contexto para un LLM a partir de las 10 mejores recetas devueltas por el sistema de puntuación.
    Para cada fila/receta genera un bloque con:
      - Título
      - Ingredientes formateados (con `format_ingredients`)
      - Instrucciones normalizadas y recortadas (con `format_directions`)
      - Señales numéricas (métricas) redondeadas a 3 decimales:
        jaccard, DRC_coverage, missing_penalization, effort

    El resultado une todos los bloques listos para pasarlo al prompt del LLM.

    Parámetros
    ----------
    top10_df : pd.DataFrame
        DataFrame ordenado por ranking previamente (aquí se está asumiendo que es top 10).

    Devuelve
    -------
    str
        Un único string con 10 bloques (o tantos como filas tenga el DataFrame), separados
        por líneas "---", adecuado como contexto de entrada para el LLM.
    """
    blocks = []
    for _, row in top10_df.iterrows():
        title_text = str(row.get("title", "")).strip()
        ingredients_text = format_ingredients(row.get("ingredients"))
        directions_text = format_directions(row.get("directions"), max_chars=900)

        jaccard = float(row.get("jaccard_score", 0.0))
        drc_cov = float(row.get("DRC_coverage", 0.0))
        missing_pen = float(row.get("missing_penalization", 0.0))
        effort_norm = float(row.get("Effort", 0.0))

        block = (
            f"Title: {title_text}\n"
            f"Recipe ingredients: {ingredients_text}\n"
            f"Directions: {directions_text}\n"
            f"[signals] jaccard={jaccard:.3f} drc_coverage={drc_cov:.3f} "
            f"missing_penalization={missing_pen:.3f} effort={effort_norm:.3f}"
        )
        blocks.append(block)

    return "\n\n---\n\n".join(blocks)


In [25]:
# Paso 4) Llamamos el chat de OLLAMA

def call_ollama_chat(
        model_name, 
        system_prompt_text, 
        user_prompt_text, 
        temperature=0.2, 
        num_ctx=8192):
    
    """
    Envía una conversación de 2 turnos (system + user) al servidor local de Ollama (el LLM usado en cuestión)
    y devuelve el contenido del mensaje de salida del modelo.

    El flujo
    --------
    - Construye el payload para POST /api/chat en http://localhost:11434.
    - Incluye 2 mensajes como parte de prompt engineering (los mensajes se definen posteriormente):
        1) system: se fija el comportamiento global (p. ej., “usa solo el contexto”, “responde en JSON”).
        2) user  : contiene un prompt simulando la llamade de un usuario pidiendo el contenido deseado de cierta forma.
        En este caso, el user prompt se usa más para formatear la salida ya que en la aplicación real, el usuario no manda ningún prompt.
        No interactúa directamente con el LLM si no que el LLM actúa como "juez final"
    - Ajusta opciones de generación: `temperature` (determina la aleatoriedad) y `num_ctx` (longitud ventana de contexto).

    Parámetros
    ----------
    model_name : str
        Nombre del modelo en Ollama
    system_prompt_text : str
        Mensaje de sistema (instrucciones globales).
    user_prompt_text : str
        Mensaje de usuario (tarea concreta).
    temperature : float, opcional
        Aleatoriedad de la generación (0 = más determinista).
    num_ctx : int, opcional
        Tamaño máximo de contexto en tokens

    Devuelve
    --------
    str
        Contenido textual de la respuesta del modelo
    """

    url = "http://localhost:11434/api/chat"
    payload = {
        "model": model_name,
        "messages": [
            {"role": "system", "content": system_prompt_text},
            {"role": "user",   "content": user_prompt_text}
        ],
        "options": {"temperature": temperature, "num_ctx": num_ctx},
        "stream": False
    }
    response = requests.post(url, json=payload, timeout=120)
    response.raise_for_status()
    return response.json()["message"]["content"]


In [26]:
# Paso 5) Prompt engineering para la llamada de LLM

def run_rag_over_top10_human_output(
    top10_df: pd.DataFrame,
    user_text,
    model_name = "llama3.1:8b",
    top_n_final = 3
):
    
    """
    Esta función actua como paso final para establecer el LLM RAG y une todas las funciones determinadas previamente.
    El objetivo final es devolver 3 recetas de las top 10 devueltas por el sistema de puntuación.

    Flujo
    --------
    1) Lee metadatos claves del dataframe de top n recetas(síntoma detectado, posibles
       deficiencias, ingredientes disponibles, etc.).
    2) Construye el contexto con las top 10 recetas candidatas (título, ingredientes e
       instrucciones) mediante `make_context_from_top10`.
    3) Se pasa al LLM:
       - un **system prompt** que fija reglas/criterios de evaluación,
       - un **user prompt** que especifica formato de salida y recordatorios (no inventar).
    4) Llama a `call_ollama_chat` (Ollama local) y devuelve `str` con la respuesta generada.

    Parámetros
    ----------
    top10_df : pd.DataFrame
        DataFrame con las top 10 recetas candidatas 
    user_text : 
        Texto original del usuario (descripción anímica/física) que se quiere reflejar en la salida.
    model_name : 
        Nombre del modelo en Ollama (por defecto se usa "llama3.1:8b").
    top_n_final : int
        Número de recetas finales que el LLM debe elegir (por defecto 3).

    Devuelve
    -------
    str
        Texto optimizado para un usuario final con las recetas elegidas y explicaciones, tal como lo devuelva el LLM.
    """

    first_row = top10_df.iloc[0]
    detected_symptom    = str(first_row.get("predicted_symptom", "")).strip()
    deficiency_list     = safe_to_list(first_row.get("deficiencia de nutrientes", []))
    deficiency_covered_by_ingredients  = safe_to_list(first_row.get("disponible en ingredientes", []))
    available_from_fridge  = safe_to_list(first_row.get("fridge_ingredients_available", []))
    available_ingredients = sorted({ing.lower() for ing in available_from_fridge})
    recipe_instructions = first_row.get('directions')
    recipe_title = first_row.get('title')
    recipe_ingredients = first_row.get('recipe_ingredients')

    context_text = make_context_from_top10(top10_df)

    system_prompt_text = (
        f"""
        Eres un asistente culinario y nutricional que se encargará de escoger las {top_n_final} mejores recetas para un usuario en base a los inputs del usuario y el contexto que te voy a dar.

        El input del usuario son los siguientes:
        Entrada del usuario: {user_text}
        Síntoma detectado: {detected_symptom}
        Ingredientes disponibles: {available_ingredients}

        El Contexto con las 10 recetas candidatas ({context_text}).
        Tarea: elige las {top_n_final} MEJORES recetas usando SOLO el contexto.
        Criterios:
        - Usa EXCLUSIVAMENTE la información del contexto proporcionado, no inventes nada
        - Tienes que verificar si la receta de verdad es para una comida. Es decir, descarta recetas de cremas, mascarillas y otras cosas que usan ingredientes de comida pero realmente no son para la nutrición
        - Tienes que verificar que la receta es realista para hacer, es decir que el usuario tiene bastantes ingredientes en la nevera de los requeridos por la receta
        - Tienes que verificar si la receta de verdad cubriría con la posible deficiencia nutricional asociada al síntoma detectado
        - No te olvides de dar las instrucciones de la receta {recipe_instructions}
        """

    )

    
    user_prompt_text = (
    f"""
    Acuérdate que no puedes usar ninguna información que no existe o inventártelo.
    El texto que me tienes devolver tiene que seguir el siguiente formato:
        - Primero, me tienes que hablar directamente, no uses tercera persona
        - Segundo, dime que síntoma me detectas en base a mi texto 
          y que deficiencias nutriocionales puede que padezca en base a {deficiency_list}. No te inventes ninguna nueva deficiencia, cita exclusivamente de {deficiency_list}
          (Asegúrate de mencionar que siempre es mejor visitar el médico si es grave) 
        - Tercero, dime los ingredientes detectados en mi nevera ({available_ingredients}) 
        - En base a esta información, recomiendame recetas:

        Para cada receta elegida, tienes que explicarme:
        - El nombre de la receta {recipe_title}
        - Los ingredientes necesarios de la receta {recipe_ingredients} (traducidos al español)
        - Cita qué alimentos o ingredientes cubren las deficiencias {deficiency_covered_by_ingredients} asociadas a {detected_symptom}. No te inventes nada
        - Menciona algunas sustituciones lógicas. Solo si el usuario no tiene algún ingrediente de la receta pero sí lo tiene en la nevera. No hace falta mencionar sustituciones de ingredientes comunes que todo el mundo tendría como agua, sal, aceite, etc.
        - Detallar las instrucciones y los pasos de la receta (información disponible en {recipe_instructions})""")

    return call_ollama_chat(
        model_name=model_name,
        system_prompt_text=system_prompt_text,
        user_prompt_text=user_prompt_text,
        temperature=0.1  # baja temperatura = más consistente
        
    )


In [27]:
rag_result = run_rag_over_top10_human_output(
    top10_df=top_10_recipes,
    user_text="me duele mucho la cabeza últimamente",
    model_name="llama3.1:8b",
    top_n_final=3
)

In [28]:
top_10_recipes

,title,ingredients,directions,NER_terms_es,recipe_ingredients,jaccard_score,missing_penalization,DRC_coverage,effort,predicted_symptom,nutritional deficiency,ingredients supplying deficiency,fridge_ingredients_available,final_score
1857177,Buffalo-Style Chik'n Salad,"[""2 frozen BOCA Spicy Chikn Veggie Patties"", ""...","[""Cook patties as directed on pkg."", ""; cut in...","['Huevos', 'apio', 'Zanahoria', 'Tomate', 'Pep...","[huevos, apio, zanahoria, tomate, pepino, queso]",0.500000,0.166667,0.20,0.011494,dolor de cabeza,magnesio; riboflavina (b2); potasio,"[aguacate, platano, patata, tomate, calabacine...","[aguacate, huevos, limon, manzana, pepino, pla...",0.273851
2166456,Market-Fresh Salad,"[""4 small boneless skinless chicken breasts (1...","[""Heat barbecue to medium-high heat."", ""Brush ...","['Carne pollo', 'Tomate', 'Lechuga/Endivia', '...","[carne pollo, tomate, lechuga/endivia, zanahor...",0.454545,0.285714,0.25,0.020764,dolor de cabeza,magnesio; riboflavina (b2); potasio,"[aguacate, platano, patata, tomate, calabacine...","[aguacate, huevos, limon, manzana, pepino, pla...",0.244839
1248534,Mexican Puffs,"[""1 None avocado, diced"", ""1 None tomato, seed...","[""Preheat the oven to 425\u00b0F. Line two bak...","['Aguacate', 'Tomate', 'Queso', 'pastelería', ...","[aguacate, tomate, queso, pastelería, huevos, ...",0.454545,0.285714,0.25,0.039303,dolor de cabeza,magnesio; riboflavina (b2); potasio,"[aguacate, platano, patata, tomate, calabacine...","[aguacate, huevos, limon, manzana, pepino, pla...",0.242985
1266234,Slimmer'S Lunch,"[""1 None hard-boiled egg, halved"", ""3.5 oz low...","[""Arrange all ingredients in an airtight lunch...","['Huevos', 'Queso', 'Tomate', 'apio', 'Zanahor...","[huevos, queso, tomate, apio, zanahoria, remol...",0.454545,0.285714,0.20,0.004079,dolor de cabeza,magnesio; riboflavina (b2); potasio,"[aguacate, platano, patata, tomate, calabacine...","[aguacate, huevos, limon, manzana, pepino, pla...",0.234008
1769104,Mediterranean English Muffin Sandwich,"[""12 small carrot, grated"", ""12 small cucumber...","[""Mix the carrots, cucumbers and tomatoes toge...","['Zanahoria', 'Pepino', 'Tomate', 'hummus', 'Q...","[zanahoria, pepino, tomate, hummus, queso, agu...",0.454545,0.285714,0.20,0.019281,dolor de cabeza,magnesio; riboflavina (b2); potasio,"[aguacate, platano, patata, tomate, calabacine...","[aguacate, huevos, limon, manzana, pepino, pla...",0.232488
666717,Turkey Cobb Salad,"[""3 oz. Backyard Grill turkey breast, diced"", ...","[""Layer lettuce, tomatoes, cucumber, egg and d...","['Carne pavo', 'Tomate', 'Queso', 'Pepino', 'H...","[carne pavo, tomate, queso, pepino, huevos]",0.400000,0.200000,0.25,0.007045,dolor de cabeza,magnesio; riboflavina (b2); potasio,"[aguacate, platano, patata, tomate, calabacine...","[aguacate, huevos, limon, manzana, pepino, pla...",0.231796
2102536,Vegie Sailboats! (--Tasty Dish--),"[""6 pickling cucumbers, cut in half lengthwise...","[""BOAT: Fill the little sailboat (ex: pickling...","['Pepino', 'Tomate', 'pimientos amarillos', 'H...","[pepino, tomate, pimientos amarillos, huevos, ...",0.454545,0.285714,0.20,0.029663,dolor de cabeza,magnesio; riboflavina (b2); potasio,"[aguacate, platano, patata, tomate, calabacine...","[aguacate, huevos, limon, manzana, pepino, pla...",0.231449
1086938,Ww 5 Points - Shrimp Louis Salad,"[""1 cup cottage cheese"", ""1/4 cup tomato juice...","[""In blender combine cottage cheese, tomato ju...","['Queso', 'Tomate', 'Huevos', 'mostaza', 'Pepi...","[queso, tomate, huevos, mostaza, pepino, gamba...",0.416667,0.375000,0.30,0.025213,dolor de cabeza,magnesio; riboflavina (b2); potasio,"[aguacate, platano, patata, tomate, calabacine...","[aguacate, huevos, limon, manzana, pepino, pla...",0.224562
1684146,Katie's Favorite Salad,"[""mesclun"", ""1 medium diced tomato"", ""1 medium...","[""Toss ingredients and serve.""]","['Tomate', 'Pepino', 'Aguacate', 'Nueces', 'Qu...","[tomate, pepino, aguacate, nueces, queso]",0.400000,0.200000,0.20

In [29]:
print(rag_result)

Me duele mucho la cabeza últimamente. El síntoma detectado es dolor de cabeza y las deficiencias nutricionales asociadas pueden ser:

* Deficiencia en vitaminas B6, B9 (ácido fólico) y magnesio.
* Posible déficit en ácidos grasos omega-3.

Los ingredientes que tengo disponibles son: ['aguacate', 'huevos', 'limon', 'manzana', 'pepino', 'platano', 'queso', 'tomate', 'yogur', 'zanahoria']

Basándome en esta información, te recomiendo las siguientes recetas:

**Receta 1: Market-Fresh Salad**

* Nombre de la receta: Salada fresca del mercado
* Ingredientes necesarios: ['huevos', 'apio', 'zanahoria', 'tomate', 'pepino']
* Alimentos o ingredientes que cubren las deficiencias asociadas al dolor de cabeza:
 + Huevos: ricos en vitaminas B6 y magnesio.
 + Zanahoria: buena fuente de vitamina B9 (ácido fólico).
* Sustituciones lógicas: si no tienes apio, puedes usar pepino como sustituto.
* Instrucciones:
 1. Calienta la parrilla a fuego medio-alto. 
 2. Pinta los huevos con 2 cucharadas de dressin